# A tour of python profiling and optimization (Part II)

*Antonino Ingargiola*, EuroScipy 2018

> premature optimization is the root of all evil (or at least most of it) in programming. 

                                                         - Donald Knuth

# Parallelism with Joblib

> [Joblib](https://joblib.readthedocs.io/) provides a *incredibly simple API* for both 
> **thread-based** a **process-based** parallelism (it also offer memoization).


## OU process function to be parallelized

*Continuing the Ornestein-Uhlenbeck process example from [Optimizing python code](Optimizing%20python%20code.ipynb).*

We define both Cython and Numba functions to be parallelized:

In [1]:
import time, os
import numpy as np
import numba
import math
from randomgen import RandomGenerator, Xoroshiro128

In [2]:
%load_ext cython

In [3]:
%%cython

import numpy as np
cimport numpy as np
from libc.math cimport exp, sqrt
import cython

@cython.boundscheck(False)
def OU_process_cy3(int num_steps, *, double[:] N_norm, double dt=0.1, 
                   double x0=0, double tau=1, double sigma=2):
    cdef Py_ssize_t i
    cdef double relax, diffuse
    cdef double[:] x = np.empty(num_steps + 1)
    
    with nogil:
        relax = exp(-dt / tau)
        diffuse = sigma * sqrt(1 - relax**2)
        x[0] = x0
        for i in range(num_steps):
            x[i+1] = x[i] * relax + diffuse * N_norm[i]
    return np.asfarray(x)

In [4]:
@numba.jit(nogil=True)
def OU_process_nb(num_steps, *, N_norm, δt=0.1, x0=0., τ=1., σ=2.):
    relax = math.exp(-δt / τ)
    diffuse = σ * math.sqrt(1 - relax**2)

    x = np.empty(num_steps + 1)
    x[0] = x0
    for i in range(num_steps):
        x[i+1] = x[i] * relax + diffuse * N_norm[i]
    return x

> **NOTE**: Both functions release the GIL! Can you find where?

# Joblib examples

*Reference: [Embarrassingly parallel for loops](https://joblib.readthedocs.io/en/latest/parallel.html)*

In [5]:
from joblib import Parallel, delayed

## Parallel sqrt

Example from the docs:

In [6]:
[math.sqrt(i ** 2) for i in range(10)]

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [7]:
Parallel(n_jobs=2)(delayed(math.sqrt)(i ** 2) for i in range(10))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

## Busy sleep

In [8]:
def verbose_sleep(n):
    print(f'PID: {os.getpid()} - busy sleeping', flush=True)
    time.sleep(n)
    print(f'PID: {os.getpid()} - done sleeping', flush=True)
    return n

In [9]:
%%timeit -n1 -r1
[verbose_sleep(5) for i in range(4)]

PID: 53583 - busy sleeping
PID: 53583 - done sleeping
PID: 53583 - busy sleeping
PID: 53583 - done sleeping
PID: 53583 - busy sleeping
PID: 53583 - done sleeping
PID: 53583 - busy sleeping
PID: 53583 - done sleeping
20 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -n1 -r1
Parallel(n_jobs=4)(delayed(verbose_sleep)(5) for i in range(4))

5.24 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Random Numbers Generations

First the single-core version:

In [11]:
num_steps = int(1e7)
num_sim = 8

rg = RandomGenerator(Xoroshiro128())

In [12]:
%%timeit
np.random.randn(num_sim*num_steps)

3 s ± 356 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Here we use thread-based parallelism (`prefer='threads'`):

In [13]:
%%timeit
Parallel(n_jobs=4, prefer='threads')(delayed(np.random.randn)(num_steps) 
                                     for i in range(num_sim))

2.69 s ± 65.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
rg.randn(num_sim*num_steps)

1.29 s ± 69.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
Parallel(n_jobs=4, prefer='threads')(delayed(rg.randn)(num_steps) 
                                     for i in range(num_sim))

1.12 s ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Process-based parallelism, commented because is too slow:

In [16]:
# %%timeit -n1 -r1
# Parallel(n_jobs=4)(delayed(np.random.randn)(num_steps) for i in range(num_sim))

In [17]:
# %%timeit -n1 -r1
# Parallel(n_jobs=4)(delayed(rg.randn)(num_steps) for i in range(num_sim))

# Parallelize the OU process

In the previous notebook we have split the simulation in
RNG and the deterministic computations. This is allows better
optimization and to easily switch to the more performant `randomgen` 
instead of `numpy.random`.

For parallelization purposes, here we define 2 wrapper function 
performing the full simulation (cython and numba versions):

In [18]:
def ou_process_cy_mp(num_steps, *, dt=0.1, x0=0, tau=1, sigma=2):
    rg = RandomGenerator(Xoroshiro128())
    N_norm = rg.randn(num_steps)
    x = OU_process_cy3(num_steps, N_norm=N_norm, dt=dt, x0=x0, tau=tau, sigma=sigma)
    return x

def ou_process_nb_mp(num_steps, *, dt=0.1, x0=0, tau=1, sigma=2):
    rg = RandomGenerator(Xoroshiro128())
    N_norm = rg.randn(num_steps)
    x = OU_process_nb(num_steps, N_norm=N_norm, δt=dt, x0=x0, τ=tau, σ=sigma)
    return x

Test calling the functions:

In [19]:
N_norm = rg.randn(num_steps)

[OU_process_cy3(num_steps, N_norm=N_norm) for _ in range(num_sim)]
[ou_process_cy_mp(num_steps) for _ in range(num_sim)]
[ou_process_nb_mp(num_steps) for _ in range(num_sim)]

[array([ 0.        , -0.91381581, -1.21480702, ...,  0.45823585,
         0.75006904,  1.25035375]),
 array([ 0.        , -0.73593378, -1.57885256, ...,  1.85407006,
         1.85068165,  1.87429777]),
 array([0.        , 0.75570866, 2.20825857, ..., 3.09085206, 1.9253846 ,
        2.78668992]),
 array([ 0.        , -0.62786681, -0.33992795, ..., -1.11356392,
        -2.80165104, -4.24673714]),
 array([ 0.        , -0.33603444, -0.29430216, ...,  1.59806292,
         3.0686767 ,  2.97526084]),
 array([ 0.        , -0.84628112, -0.06544085, ...,  1.25197344,
         2.72034133,  2.59642877]),
 array([ 0.        ,  0.43212653,  0.27428987, ...,  0.28326639,
        -1.03328842, -1.7883494 ]),
 array([ 0.        , -0.40535071, -0.93833953, ...,  1.58546064,
         1.08065149, -0.17701509])]

# Parallelize Cython 

We need to release the GIL. How?

- cython functions only callable from C (`cdef`) can release the gil using
      cdef function_name() nogi:
- cython functions callable from python (`def`, `cpdef`) 
  can release the gil using a `with nogil:` block.
  
Since our function needs to be called from python, we use the second approach.

In [20]:
num_steps = int(1e7)
num_sim = 8

### Parallel cython: OU process

In [21]:
%%timeit
[OU_process_cy3(num_steps, N_norm=N_norm) for _ in range(num_sim)]

545 ms ± 28.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
Parallel(n_jobs=4, prefer='threads')(delayed(OU_process_cy3)(num_steps, N_norm=N_norm) 
                                     for _ in range(num_sim))

300 ms ± 41.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Parallel cython: OU process + RNG

In [23]:
%%timeit
[ou_process_cy_mp(num_steps) for _ in range(num_sim)]

1.9 s ± 247 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
Parallel(n_jobs=4, prefer='threads')(delayed(ou_process_cy_mp)(num_steps) 
                                     for _ in range(num_sim))

966 ms ± 195 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Results

- We get a **~2x** speed improvement on a 2-core system 
- Empirically, best performance with 4 threads on 2-core system

# Parallelize Numba

In [25]:
num_steps = int(1e7)
num_sim = 8

### Parallel Numba: OU process

In [26]:
%%timeit
[OU_process_nb(num_steps, N_norm=N_norm, δt=0.1, x0=0, σ=1, τ=1)
 for _ in range(num_sim)]

460 ms ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
Parallel(n_jobs=4, prefer='threads')(delayed(OU_process_nb)(num_steps, N_norm=N_norm, δt=0.1, x0=0, σ=1, τ=1)
                                     for _ in range(num_sim))

279 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Parallel Numba: OU process + RNG

In [28]:
%%timeit
[ou_process_nb_mp(num_steps) for _ in range(num_sim)]

1.41 s ± 49.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
Parallel(n_jobs=4, prefer='threads')(delayed(ou_process_nb_mp)(num_steps) 
                                     for _ in range(num_sim))

969 ms ± 206 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Results

- We get a **>2x** speed improvement on a 2-core system 
- Empirically, best performance with 4 threads on 2-core system

# Take-home message

## Thread-based parallelism

*Release the GIL to unlock a performance boost with thread-based parallelism.*

Numpy's functions (including `randomgen`) already release the GIL! 🎉🎉🎉

## Process-based parallelism

Rarely achieves gains unless the application is heavily CPU-bound
and has relatively low RAM usage.

# Standard lib: Threading and Multiprocessing

The `mutiprocessing` package provides a relatively simple API (`Pool`)
compared to the `threading` package.

Unfortunately, even using `mutiprocessing` is much more cunbersome that joblib.

In [30]:
from multiprocessing import Pool

In [31]:
%%timeit -n1 -r1
with Pool(processes=4) as pool:
    try:
        Res = []
        for i in range(num_sim):
            result = pool.apply_async(
                ou_process_cy_mp, kwds=dict(num_steps=num_steps, 
                                            dt=0.1, x0=0, 
                                            tau=1, sigma=2))
            Res.append(result)
        X = []
        for i, result in enumerate(Res):
            X.append(result.get())
            #print(f'[Worker {i}] returned an array of shape {len(X[-1])}', flush=True)
    except KeyboardInterrupt:
        print('\n>>> Got keyboard interrupt.\n', flush=True)
print('Closing subprocess pool.', flush=True)

Closing subprocess pool.
15.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


The `threading` API is even more complex.

## Lessons

- Joblib's API rocks!